In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import cv2
from PIL import Image

In [2]:
import os
import cv2

images_w_waldo = []
# CONFIG
input_image_dir = "photos/initial"
input_label_dir = "photos/initial_labels"
output_image_dir = "output/images"
output_label_dir = "output/labels"
split_size = 64  # tile size
class_id = 1  # Waldo's class ID in original labels

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

def read_pixel_label(label_path):
    boxes = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    cls, cx, cy, w, h = map(float, parts)
                    boxes.append((int(cls), cx, cy, w, h))
    return boxes

def convert_to_box_edges(box):
    cls, cx, cy, w, h = box
    x1 = int(cx-w/2)
    y1 = int(cy-h/2)
    x2 = int(cx + w/2)
    y2 = int(cy + h/2)
    return cls, x1, y1, x2, y2

def is_box_in_tile(bx1, by1, bx2, by2, tx1, ty1, tx2, ty2, margin=8):
    # Expand tile bounds by margin
    tx1 -= margin
    ty1 -= margin
    tx2 += margin
    ty2 += margin

    return not (bx2 <= tx1 or bx1 >= tx2 or by2 <= ty1 or by1 >= ty2)

def adjust_box_to_tile(cls, bx1, by1, bx2, by2, tx1, ty1, tile_w, tile_h):
    print(cls, bx1, by1, bx2, by2, tx1, ty1, tile_w, tile_h)
    print("!!!!!!!!!!!!!!!!!!!")
    new_x1 = max(bx1, tx1)
    print(new_x1)
    new_y1 = max(by1, ty1)
    new_x2 = min(bx2, tx1 + tile_w)
    print(new_x2)
    new_y2 = min(by2, ty1 + tile_h)

    bw = new_x2 - new_x1
    bh = new_y2 - new_y1
    if bw <= 0 or bh <= 0:
        return None

    cx = new_x1 + bw / 2 - tx1
    cy = new_y1 + bh / 2 - ty1
    print(cx)
    return cls, cx, cy, bw, bh

image_count = 0

for img_name in os.listdir(input_image_dir):
    if not img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    img_path = os.path.join(input_image_dir, img_name)
    label_path = os.path.join(input_label_dir, os.path.splitext(img_name)[0] + ".txt")

    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Skipping unreadable image: {img_name}")
        continue

    img_h, img_w = img.shape[:2]

    boxes = read_pixel_label(label_path)
    print(label_path)
    print(boxes)
    pixel_boxes = [b for b in boxes if b[0] == class_id]
    print(pixel_boxes)


    for y in range(0, img_h, split_size):
        
        for x in range(0, img_w, split_size):
            tile = img[y:y+split_size, x:x+split_size]
            if tile.shape[0] != split_size or tile.shape[1] != split_size:
                
                continue
                

            tile_filename = f"{image_count:05d}.jpg"
            tile_label_filename = f"{image_count:05d}.txt"
            image_count += 1

            waldo_found = False
            new_labels = []

            tx1, ty1 = x, y
            tx2, ty2 = x + split_size, y + split_size
            # print("hello)")
            # print(pixel_boxes)
            for cls, bx1, by1, bx2, by2 in pixel_boxes:
                # print(tx1, ty1, tx2, ty2)
                if is_box_in_tile(bx1, by1, bx2, by2, tx1, ty1, tx2, ty2):
                    adj_box = adjust_box_to_tile(cls, bx1, by1, bx2, by2, tx1, ty1, split_size, split_size)
                    print(y, adj_box)
                    print("yes")
                    if adj_box:
                        waldo_found = True
                        new_labels.append(adj_box)

            cv2.imwrite(os.path.join(output_image_dir, tile_filename), tile)

            with open(os.path.join(output_label_dir, tile_label_filename), "w") as f:
                if waldo_found:
                    for lbl in new_labels:
                        f.write(f"{lbl[0]} {lbl[1]:.2f} {lbl[2]:.2f} {lbl[3]:.2f} {lbl[4]:.2f}\n")
                        images_w_waldo.append(tile_label_filename)
                        
                
                else:
                    f.write("0\n")
                

print(f"✅ Finished splitting and labeling. Total tiles: {image_count}")
print(images_w_waldo)


photos/initial_labels\wally1.txt
[(1, 717.0, 521.0, 737.0, 563.0)]
[(1, 717.0, 521.0, 737.0, 563.0)]
1 717.0 521.0 737.0 563.0 704 512 64 64
!!!!!!!!!!!!!!!!!!!
717.0
737.0
23.0
512 (1, 23.0, 30.0, 20.0, 42.0)
yes
photos/initial_labels\wally10.txt
[(1, 176.0, 160.0, 190.0, 182.0)]
[(1, 176.0, 160.0, 190.0, 182.0)]
1 176.0 160.0 190.0 182.0 128 128 64 64
!!!!!!!!!!!!!!!!!!!
176.0
190.0
55.0
128 (1, 55.0, 43.0, 14.0, 22.0)
yes
1 176.0 160.0 190.0 182.0 192 128 64 64
!!!!!!!!!!!!!!!!!!!
192
190.0
128 None
yes
photos/initial_labels\wally11.txt
[(1, 466.0, 1534.0, 480.0, 1556.0)]
[(1, 466.0, 1534.0, 480.0, 1556.0)]
1 466.0 1534.0 480.0 1556.0 448 1472 64 64
!!!!!!!!!!!!!!!!!!!
466.0
480.0
25.0
1472 (1, 25.0, 63.0, 14.0, 2.0)
yes
1 466.0 1534.0 480.0 1556.0 448 1536 64 64
!!!!!!!!!!!!!!!!!!!
466.0
480.0
25.0
1536 (1, 25.0, 10.0, 14.0, 20.0)
yes
photos/initial_labels\wally12.txt
[(1, 1370.0, 639.0, 1385.0, 663.0)]
[(1, 1370.0, 639.0, 1385.0, 663.0)]
1 1370.0 639.0 1385.0 663.0 1344 576 64 64


In [3]:

# CONFIG
image_path = "output/images/01071.jpg"       # Your new image path
label_path = "output/labels/01071.txt"       # Corresponding label file
save_path = "output/vis/00001_vis.jpg"       # Optional: Save the visualized image

# Create output folder if saving
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Read image
image = cv2.imread(image_path)
if image is None:
    raise ValueError("Image not found or unreadable.")

# Read label
with open(label_path, 'r') as f:
    lines = f.readlines()

for line in lines:
    parts = line.strip().split()
    if len(parts) == 1 and parts[0] == '0':
        continue  # No Waldo in this tile

    cls_id, x, y, w, h = map(float, parts)

    # Coordinates are in pixels
    x1 = int(x)-int(w/2)
    y1 = int(y)-int(h/2)
    y2 = int(h)+int(h/2)
    x2 = int(x)+int(w/2)

    print(x2, y2)

    # Draw rectangle and class label
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0,0), 2)
    cv2.putText(image, f"Class {int(cls_id)}", (x1, max(10, y1 - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

# Save image
cv2.imwrite(save_path, image)
print(f"✅ Saved labeled image to: {save_path}")


✅ Saved labeled image to: output/vis/00001_vis.jpg
